In [76]:
import requests
from massive import RESTClient
from datetime import datetime, timedelta
import numpy as np
import pandas as pd
from datetime import date

In [115]:
class Analyze:
    """
        Makes a dictionary for each day and gives RSI, price value, and number of transactions/shares 
        traded
    """

    def __init__(self, client, day_range, ticker):
        self.client = client
        self.today = datetime.today().strftime('%Y-%m-%d')
        self.day_range = day_range
        self.ticker = ticker

    """
        Converts milliseconds timestamp to more readable format
    """
    def convert_timestamp(self, ts):
        return datetime.fromtimestamp(ts/1000).strftime('%Y-%m-%d')

    """
        Gets RSI values over time frame and for a specific ticker, also 
        gets the minimum date, as this will give me the last 365 trading days
    """
    def get_rsi(self, timespan='day', adjusted=True, window=14, series_type='close', order='desc'):
        rsi_data = self.client.get_rsi(
            ticker=self.ticker,
            timespan=timespan,
            adjusted=adjusted,
            window=window,
            series_type=series_type,
            order=order,
            limit=self.day_range
        )
        self.from_date = None
        self.time_dict = {}
        for _ in rsi_data.values:
            timestamp = self.convert_timestamp(_.timestamp)
            # sets the from date to the earliest date in the RSI data
            if pd.isna(self.from_date) or (self.from_date and timestamp < self.from_date):
                self.from_date = timestamp
            self.time_dict[timestamp] = {
                'rsi': _.value
            }
    """
        Gets prices for a specific ticker over a set time
    """
    def get_price(self, multiplier=1, timespan='day'):
        # set close, as I want to add a change in close price for each day
        last_close = None

        for _ in self.client.list_aggs(
                ticker=self.ticker, 
                multiplier=multiplier, 
                timespan=timespan, 
                from_=self.from_date, 
                to=self.today
        ):
            timestamp = self.convert_timestamp(_.timestamp)
            if not pd.isna(last_close):
                change = ((_.close - last_close) / last_close) * 100
                last_close = _.close
            else:
                change = None
                last_close = _.close
            if timestamp in self.time_dict:
                self.time_dict[timestamp] = {
                    'open': _.open, 
                    'close': _.close, 
                    'percent_close_change': change,
                    'volume': _.volume, 
                    'high': _.high, 
                    'low': _.low,
                    'transactions': _.transactions
                }
            else:
                print(f"Date {timestamp} not in Price data, but used in RSI")

    def run(self):
        self.get_rsi()
        self.get_price()
        return pd.DataFrame.from_dict(self.time_dict, orient='index')

In [117]:
"""
    use api key and set up client
"""
api_key = 'hJQGYwLtVkOmM45yOBeEOg99YqOzJAz9'
client = RESTClient(api_key=api_key)

ticker = 'AAPL'

"""
    instance of class
"""
inst = Analyze(client=client, ticker='AAPL', day_range=365)
# if want to work with df
df = inst.run()
# if want to work with dict
dict_data = inst.time_dict

In [118]:
df

,open,close,percent_close_change,volume,high,low,transactions
2025-11-07,269.795,268.47,-0.481892,48227365.0,272.290,266.770,754451
2025-11-06,267.890,269.77,-0.136966,51203925.0,273.400,267.890,718677
2025-11-05,268.610,270.14,0.037032,43644270.0,271.700,266.930,587713
2025-11-04,268.325,270.04,0.367961,49271745.0,271.486,267.615,713268
2025-11-03,270.420,269.05,-0.488220,50194063.0,270.850,266.250,731847
...,...,...,...,...,...,...,...
2024-06-03,192.900,194.03,0.925878,50080539.0,194.990,192.520,700957
2024-05-31,191.440,192.25,0.501856,75158277.0,192.570,189.910,618230
2024-05-30,190.760,191.29,0.525514,49947941.0,192.180,190.630,568984
2024-05-29,189.610,190.29,0.157903,53068016.0,192.247,189.510,603502
